In [1]:
import psycopg2;
from flask import Flask, jsonify, request
from psycopg2 import sql;
from flask_cors import CORS

In [2]:
# setup local development server
app = Flask(__name__)

CORS(app)  # Enable CORS for all routes

# API root message
@app.route('/', methods=['GET'])
def index():
    return 'Welcome to the API!'

# NFT historical data call
@app.route('/api/nft_data', methods=['GET'])
def get_data():
    # Get the 'nft_ids' parameter from the request URL
    nft_ids = request.args.getlist('nft_ids')
    
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host="localhost",
        database="joostdelaere",
        user="joostdelaere",
        password=""
    )

    cursor = conn.cursor()
    
    # Construct the SQL query with the nft_ids filter
    query = sql.SQL("SELECT * FROM user_info WHERE nft_id IN ({})").format(
        sql.SQL(',').join(sql.Placeholder() * len(nft_ids))
    )
    
    # Execute the query with the nft_ids as parameters
    cursor.execute(query, nft_ids)

    # Fetch the data from the user_info table
    data = cursor.fetchall()

    # Convert the fetched data to a list of dictionaries
    results = []
    for row in data:
        result = {
            'nft_id': row[1],
            'timestamp': str(row[3]),
            'staking_rewards': float(row[5]),
            'interest_earnings': float(row[6])
        }
        results.append(result)

    # Close the database connection
    cursor.close()
    conn.close()
    
    # Return the fetched data as JSON
    return jsonify(results)

# run development server
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Jul/2023 10:18:55] "GET /api/nft_data?nft_ids=%231%23&nft_ids=%232%23&nft_ids=%233%23 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 10:22:20] "GET /api/nft_data?nft_ids=%231%23&nft_ids=%232%23&nft_ids=%233%23 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 10:22:52] "GET /api/nft_data?nft_ids=%231%23&nft_ids=%232%23&nft_ids=%233%23'%20from%20origin%20'http://localhost:5173 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2023 11:14:48] "GET /api/nft_data?nft_ids=%231%23&nft_ids=%232%23&nft_ids=%233%23 HTTP/1.1" 200 -
